<a href="https://colab.research.google.com/github/cateto/python4NLP/blob/main/kobert/news_classifications_pytorch_kobert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
# 변경: 최신 버전으로 설치하면 "Input: must be Tensor, not str" 라는 에러 발생
!pip install transformers==3
!pip install torch

     |████████████████████████████████| 46.9 MB 41 kB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 344 kB 37.7 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595727 sha256=cb0d23f3772c662cf9c1483e33e2e25dfdbd603e9681b7ed4c1726631304bf30
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
     |████████████████████████████████| 1.2 MB 31.7 MB/s 
     |████████████████████████████████| 754 kB 36.9 MB/s 
     |████████████████████████████████| 895 kB 55.2 MB/s 
     |████████████████████████████████| 3.0 MB 50.9 MB/s 


In [2]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-b_ook68f
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-b_ook68f
  Created wheel for kobert: filename=kobert-0.1.2-py3-none-any.whl size=12770 sha256=a6bfc961ad429ffe5657ce3f147d8ad640cffa85f6b4a3c437bd37b4149483cd
  Stored in directory: /tmp/pip-ephem-wheel-cache-gcvnlq6r/wheels/d3/68/ca/334747dfb038313b49cf71f84832a33372f3470d9ddfd051c0
Successfully built kobert


In [3]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [4]:
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

In [5]:
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [6]:
##GPU 사용 시
device = torch.device("cuda:0")

In [7]:
bertmodel, vocab = get_pytorch_kobert_model()

[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]


In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import pandas as pd
dataset = pd.read_csv('/content/drive/MyDrive/rsn_nlp_project/sample_0727.csv')

#라벨 데이터 category 정수로 변환

encoder = LabelEncoder()
encoder.fit(dataset['category'])
dataset['category'] = encoder.transform(dataset['category'])

# possible_labels = dataset['category'].unique()
# label_dict = {}
# for index, possible_label in enumerate(possible_labels):
#     label_dict[possible_label] = index

# dataset['category'] = dataset['category'].replace(label_dict)

#이어서 contents 전처리(정제)
dataset['contents'] = dataset['contents'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]"," ")
dataset['contents'] = dataset['contents'].str.replace("[ ]{2,}"," ")

dataset.head()

,id,contents,category,date
0,NLRW1900000161.1,명 게놈 해독 무병장수 새 장 연다 울산 게놈코리아 프로젝트 본격 가동 행복생활권...,4,20170104
1,NLRW1900000161.2,현대차 아이오닉 야간 자율주행 성공 라스베이거스 도심 거뜬 기간 첫 시연 복잡한 ...,1,20170104
2,NLRW1900000161.3,울산 미래 환경산업의 허브를 기대하며 우리는 언뜻 환경 분야라고 하면 영리를 추구...,0,20170104
3,NLRW1900000161.4,파사현정 이즈음 온 나라 안이 벌집을 쑤신 듯 소란스럽다 사이비종교 교주의 딸 최...,5,20170104
4,NLRW1900000161.5,현대식 시장 건립 언양알프스시장 개 점포 수용 울주군 언양시가지 교통망 개선사업 ...,5,20170104


In [13]:
mapping = dict(zip(range(len(encoder.classes_)), encoder.classes_))
mapping

{0: 'IT/과학',
 1: '경제',
 2: '문화',
 3: '미용/건강',
 4: '사회',
 5: '생활',
 6: '스포츠',
 7: '연예',
 8: '정치'}

In [101]:
# test data / train data 나누기

train, test = train_test_split(dataset, test_size=0.2, random_state=42)
print("훈련용 data 개수:", len(train))
print("테스트용 data 개수:", len(test))
train.to_csv('/content/drive/MyDrive/rsn_nlp_project/sample_train.txt', sep = '\t' , index = False)
test.to_csv('/content/drive/MyDrive/rsn_nlp_project/sample_test.txt', sep = '\t' , index = False)

훈련용 data 개수: 4574
테스트용 data 개수: 1144


재시도

In [102]:
dataset_train = nlp.data.TSVDataset("/content/drive/MyDrive/rsn_nlp_project/sample_train.txt", field_indices=[1,2], num_discard_samples=1)
dataset_test = nlp.data.TSVDataset("/content/drive/MyDrive/rsn_nlp_project/sample_test.txt", field_indices=[1,2], num_discard_samples=1)

In [104]:
dataset_train[:5]

[[' 울산지역 소프트웨어 기업 유시스 기활법 승인 스마트 공장 무인항공기 솔루션 등 고부가가치 신산업 진출 기반 산업 드론분야 입지 탄탄 국토부 시범사업자 선정 수출유망기업 해외판로 개척 나서 산자부 곳 신규 누적 건 승인 서비스업 기업 사업재편 본격화 울산지역의 소프트웨어 기업인 유시스가 기업활력법 승인을 받는데 성공해 드론과 스마트공장 등 고부가가치 신산업 진출에 날개를 달게 됐다 산업통상자원부는 기업활력법에 따라 유시스 엔지니어링 등 개 기업의 사업재편계획을 신규 승인했다고 일 밝혔다 기업활력법은 정상 기업의 자율적 사업재편을 돕는 법으로 상법 세법 공정거래법 등 관련 절차와 규제를 간소화해주고 패키지로 여러 정책 지원을 해줘 원샷법 으로 불린다 울산 남구 신정동에 본사를 둔 유시스는 국내 조선 업체에 해양플랜트 설계 프로그램 선박 자동화 프로그램 등 소프트웨어 를 공급하는 기업이다 이 회사는 기활법 승인으로 보유하고 있는 조선 설계 소프트웨어의 지식재산권을 일부 매각하고 스마트공장 무인항공기 솔루션과 하드웨어 제작 사업에 새롭게 진출한다 유시스가 진출하는 스마트공장 솔루션은 사물인터넷 기술을 기반으로 한 스마트공장 솔루션이고 무인항공기 솔루션은 무인비행체를 운영하는 시스템이다 하드웨어 제작 사업은 스마트공장 시스템 센서 디바이스 무인비행기 기체 프레임 및 컨트롤러 등이다 유시스는 년 관련 하드웨어와 소프트웨어 유통업체로 설립된 씨에프정보통신에서 출발해 조선분야를 비롯 각종 소프트웨어를 개발하면서 성장했으며 년 상호를 변경했다 수년 전부터는 드론산업에 뛰어들어 사물인터넷 를 기반으로 산업용 드론분야에서 입지를 다져가고 있다 최근 울산과학기술원 와 국토교통부의 드론시범사업 추가사업자에 선정됐으며 해외 판로 개척에도 나서고 있다 년에는 미래창조과학부의 분야 유망기업에 선정됐고 중소기업청으로부터 수출유망중소기업으로 지정되기도 했다 이 기업 외에도 앞서 울산지역에서는 지난해 석유화학단지 내 가성소다 제조공장을 매각하는 한화케미칼과 이 공장을 인수하는 유니드가 

In [105]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model


In [106]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [107]:
## Setting parameters
max_len = 64 # 해당 길이를 초과하는 단어에 대해선 bert가 학습하지 않음
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate = 5e-5

In [108]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [110]:
data_train[0]

(array([   2, 3525, 7337, 2852, 1292, 3574, 6705, 6664, 1258, 7948, 6335,
        2957, 2931, 1051, 2095, 7119, 7847, 5561, 2868, 1815,  993, 6399,
        5330, 7483, 3010, 6517, 4369, 1278, 2641, 1788, 6084, 6422, 3849,
        4707, 7590, 1160, 2977, 6506, 2761, 2906, 7063, 6165, 5577, 5012,
        7688, 6079,  853, 1381, 2640, 7147, 6398, 1021, 3014, 1533,  881,
        2957, 2724, 6873, 1292, 2609, 7191, 7720, 2412,    3], dtype=int32),
 array(64, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       dtype=int32),
 0)

In [111]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [112]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                #  num_classes=2,
                 num_classes=9,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [113]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [114]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [115]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [116]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [117]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [118]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [119]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 1 batch id 1 loss 2.42352032661438 train acc 0.078125

epoch 1 train acc 0.40360243055555556


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



epoch 1 test acc 0.7135416666666666


...


epoch 3 test acc 0.8195684523809523


epoch 4 batch id 1 loss 0.36275333166122437 train acc 0.921875

epoch 4 train acc 0.9279513888888888



epoch 4 test acc 0.8294890873015873


epoch 5 batch id 1 loss 0.17425808310508728 train acc 0.984375

epoch 5 train acc 0.9600694444444444



epoch 5 test acc 0.8302331349206349


In [152]:
# 테스트 문장 예측

test_sentence = '태권도 대표팀이 대회 마지막 날 종주국의 면모를 보였다. 비록 ‘노골드’라는 아쉬운 성적을 남겼지만 경기 종료 1초 전 ‘버저비터 발차기’로 태권도의 재미를 높였고, 암을 극복하고 메달을 따내는 ‘인간 승리’의 드라마도 만들었다. 이다빈(25·서울시청)은 27일 일본 지바 마쿠하리 메세 A홀에서 열린 2020 도쿄 올림픽 태권도 여자 67㎏초과급 준결승에서 ‘버저비터 발차기’를 선보였다. 세계랭킹 1위 비안카 워크던(영국)을 맞아 22-24로 끌려가던 이다빈은 경기 종료 직전 왼발을 들어 워크던의 얼굴에 꽂아 넣으며 3점을 따내 극적인 역전승을 일궈냈다. 대회 내내 수비 위주의 경기가 반복되며 ‘발펜싱’이라는 오명을 얻었던 태권도가 극적이고 재밌는 경기라는 점을 이다빈이 준결승에서 제대로 보여줬다.'
test_label = '6'
temp_set = []
temp_set.append(test_sentence)
temp_set.append(test_label)
test_set = []
test_set.append(temp_set)

In [153]:
test_set

[['태권도 대표팀이 대회 마지막 날 종주국의 면모를 보였다. 비록 ‘노골드’라는 아쉬운 성적을 남겼지만 경기 종료 1초 전 ‘버저비터 발차기’로 태권도의 재미를 높였고, 암을 극복하고 메달을 따내는 ‘인간 승리’의 드라마도 만들었다. 이다빈(25·서울시청)은 27일 일본 지바 마쿠하리 메세 A홀에서 열린 2020 도쿄 올림픽 태권도 여자 67㎏초과급 준결승에서 ‘버저비터 발차기’를 선보였다. 세계랭킹 1위 비안카 워크던(영국)을 맞아 22-24로 끌려가던 이다빈은 경기 종료 직전 왼발을 들어 워크던의 얼굴에 꽂아 넣으며 3점을 따내 극적인 역전승을 일궈냈다. 대회 내내 수비 위주의 경기가 반복되며 ‘발펜싱’이라는 오명을 얻었던 태권도가 극적이고 재밌는 경기라는 점을 이다빈이 준결승에서 제대로 보여줬다.',
  '6']]

In [154]:
test_set = BERTDataset(test_set, 0, 1, tok, max_len, True, False)
test_input = torch.utils.data.DataLoader(test_set, batch_size=1, num_workers=9)

for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_input)):
  token_ids = token_ids.long().to(device)
  segment_ids = segment_ids.long().to(device)
  valid_length= valid_length
  out = model(token_ids, valid_length, segment_ids)
  print(out)

  model.eval() # 평가 모드로 변경
      
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
      token_ids = token_ids.long().to(device)
      segment_ids = segment_ids.long().to(device)
      valid_length= valid_length
      label = label.long().to(device)
      out = model(token_ids, valid_length, segment_ids)
      test_acc += calc_accuracy(out, label)
  print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 9 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


tensor([[-0.8341, -0.4869, -0.2035, -0.4738, -1.3645, -0.6335,  3.7560,  0.1973,
         -0.6556]], device='cuda:0', grad_fn=<AddmmBackward>)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  del sys.path[0]


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))



epoch 5 test acc 4.151165674603175

